# Upload pre-trained from timm

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model = timm.create_model('vit_base_patch16_224', pretrained=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

# Upload dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import numpy as np

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder('/content/drive/MyDrive/Project/train', transform=transform)

indices = np.random.choice(len(dataset), 3000, replace=False)
subset_dataset = Subset(dataset, indices)

batch_size = 32
data_loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
val_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_dataset = ImageFolder('/content/drive/My Drive/Project/valid', transform=val_transform)

val_indices = np.random.choice(len(val_dataset), 300, replace=False)
subset_val_dataset = Subset(val_dataset, val_indices)

val_loader = DataLoader(subset_val_dataset, batch_size=batch_size, shuffle=False)

# PEFT LoRA 적용

In [ ]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 21.4 MB/s eta 0:00:00


In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

trainable params: 86567656 || all params: 86567656 || trainable%: 100.00


In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["attn.qkv"],
    modules_to_save=["classifier"],
)

lora_model = get_peft_model(model, config)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
print_trainable_parameters(lora_model)

trainable params: 589824 || all params: 87157480 || trainable%: 0.68


# Naive Model

## Train

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
import os

num_classes = 2
model.head = nn.Linear(model.head.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

def validate(model, data_loader, criterion, device):
    model.eval()
    model.to(device)
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

model_dir = '/content/drive/MyDrive/Project/Vit_model_save'
os.makedirs(model_dir, exist_ok=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs = 10
for epoch in range(epochs):
    model.train()
    for images, labels in data_loader:
        c
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Training Loss: {loss.item()}')

    val_loss, val_accuracy = validate(model, val_loader, criterion, device)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss}, Accuracy: {val_accuracy}%')

    if (epoch + 1) % 5 == 0:
        model_name = f'Naive model_epoch_{epoch+1}.pth'
        model_path = os.path.join(model_dir, model_name)
        torch.save(model.state_dict(), model_path)
        print(f'Model saved at: {model_path}')

Epoch 1, Training Loss: 0.04712603613734245
Epoch 1, Validation Loss: 0.07327490374445915, Accuracy: 97.0%
Epoch 2, Training Loss: 0.08074439316987991
Epoch 2, Validation Loss: 0.23631146773695946, Accuracy: 91.66666666666667%
Epoch 3, Training Loss: 0.0009470694349147379
Epoch 3, Validation Loss: 0.9849343985319138, Accuracy: 81.0%
Epoch 4, Training Loss: 0.00014174157695379108
Epoch 4, Validation Loss: 0.16321686680894346, Accuracy: 95.0%
Epoch 5, Training Loss: 0.001646352931857109
Epoch 5, Validation Loss: 0.1349247632548213, Accuracy: 95.0%
Model saved at: /content/drive/MyDrive/Project/Vit_model_save/Naive model_epoch_5.pth
Epoch 6, Training Loss: 0.0006583644426427782
Epoch 6, Validation Loss: 0.14288764335215093, Accuracy: 95.66666666666667%
Epoch 7, Training Loss: 0.0001694320817478001
Epoch 7, Validation Loss: 0.3097483916208148, Accuracy: 90.33333333333333%
Epoch 8, Training Loss: 0.14643724262714386
Epoch 8, Validation Loss: 0.14175594663247465, Accuracy: 94.66666666666667%

## Import saved model

In [ ]:
base_model = timm.create_model('vit_base_patch16_224', pretrained=False)
base_model.head = nn.Linear(base_model.head.in_features, num_classes)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["attn.qkv"],
    modules_to_save=["classifier"],
)

loaded_model = get_peft_model(base_model, lora_config)

model_path = '/content/drive/MyDrive/Project/Vit_model_save/model_epoch_9.pth'
loaded_model.load_state_dict(torch.load(model_path), strict=False)
loaded_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=2304, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=2304, bias=False)
              )
              (lora_embedding_A): Parame

## Test

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
from torchvision import transforms, datasets
from tqdm import tqdm
import pandas as pd

testset_path = '/content/drive/MyDrive/Project/test'

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

testset = datasets.ImageFolder(root=testset_path, transform=transform)
test_loader = DataLoader(testset, batch_size=1, shuffle=False)

## Only LoRA with CrossEntropy

In [ ]:
if len(testset) == 500:
    results = []
    model.eval()
    model.to(device)
    for images, _ in tqdm(test_loader, total=len(test_loader)):
        images = images.to(device)
        outputs = model(images)

        if isinstance(outputs, dict) and 'logits' in outputs:
            logits = outputs['logits'].cpu()
        else:
            logits = outputs.cpu()

        _, predicted = torch.max(logits, dim=1)
        label = 'real' if predicted.item() == 1 else 'generated'
        results.append([label])

    df = pd.DataFrame(results, columns=['label'])
    results_path = '/content/drive/MyDrive/Project/test/results_naive.csv'
    df.to_csv(results_path, index=False)
    print(f'Result stored: {results_path}')

else:
    print("Number of datasets is incorrect. Number of verified data:", len(testset))

100%|██████████| 500/500 [02:42<00:00,  3.09it/s]

Result stored: /content/drive/MyDrive/Project/test/results_naive.csv


In [ ]:
def calcul_accuracy(predicted_labels, labels_csv_path):
    true_labels_df = pd.read_csv(labels_csv_path, sep=';')
    true_labels = true_labels_df['label'].values
    predicted_labels_flat = [label[0] for label in predicted_labels]

    matches = true_labels == predicted_labels_flat
    correct_predictions = matches.sum()

    print(f"Correct predictions: {correct_predictions} out of {len(matches)}")

    accuracy = correct_predictions / len(matches)

    return accuracy

In [ ]:
predicted_labels = results
labels_csv_path = '/content/drive/MyDrive/Project/test/test_labels.csv'
accuracy = calcul_accuracy(predicted_labels, labels_csv_path)
print(f"Accuracy: {accuracy*100:.2f}%")

Correct predictions: 420 out of 500
Accuracy: 84.00%


## Valid with generated image

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
import os
from PIL import Image

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

extra_val_path = '/content/drive/MyDrive/Project/GeneratedImages/generated'
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.lower().endswith(('jpg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = 0

        if self.transform:
            image = self.transform(image)

        return image, label

extra_val_dataset = CustomImageDataset(image_dir=extra_val_path, transform=val_transform)
extra_val_indices = np.random.choice(len(extra_val_dataset), 100, replace=False)
extra_subset_val_dataset = Subset(extra_val_dataset, extra_val_indices)
extra_val_loader = DataLoader(extra_subset_val_dataset, batch_size=32, shuffle=False)

def validate_1(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            logits = outputs['logits']
            loss = criterion(logits, labels)

            total_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(val_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy}%')
    return avg_loss, accuracy

model = ModelWrapper(loaded_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()

new_validation_loss, new_validation_accuracy = validate_1(model, extra_val_loader, criterion, device)

print(f'(Generated) Validation Loss: {new_validation_loss:.4f}')
print(f'(Generated) Validation Accuracy: {new_validation_accuracy:.2f}%')

# SCL Model

## SCL loss func

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os

def single_center_loss(features, labels, margin=1.0):
    real_mask = labels == 1
    generated_mask = labels == 0

    real_features = features[real_mask]
    generated_features = features[generated_mask]

    if real_features.size(0) == 0 or generated_features.size(0) == 0:
        return torch.tensor(0.0, device=features.device)

    center = real_features.mean(dim=0)
    dreal = (real_features - center).norm(dim=1).mean()
    dgenerated = (generated_features - center).norm(dim=1).mean()

    return dreal + torch.clamp(dreal - dgenerated + margin, min=0.0)

class ModelWrapper(nn.Module):
    def __init__(self, peft_model):
        super().__init__()
        self.peft_model = peft_model.base_model.model

    def forward(self, x):
        features = self.peft_model.patch_embed(x)
        features = self.peft_model.pos_drop(features)

        for block in self.peft_model.blocks:
            features = block(features)

        gap_features = torch.mean(features, dim=1)
        logits = self.peft_model.head(gap_features)

        return {'logits': logits, 'features': gap_features}

def train_SCL(model, train_loader, criterion, optimizer, device, scl_weight=0.1):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        logits = outputs['logits']
        features = outputs['features']

        ce_loss = criterion(logits, labels)
        scl_loss = single_center_loss(features, labels)
        loss = ce_loss + scl_weight * scl_loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Training Loss: {avg_loss}')
    return avg_loss

def validate_SCL(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            logits = outputs['logits']
            loss = criterion(logits, labels)

            total_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(val_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {avg_loss}, Accuracy: {accuracy}%')
    return avg_loss, accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModelWrapper(lora_model)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_loader = data_loader
valid_loader = val_loader
model_dir = '/content/drive/MyDrive/ArtificialIntelligence/Project/Vit_model_save'
os.makedirs(model_dir, exist_ok=True)

epochs = 10
scl_weight = 0.1
for epoch in range(epochs):
    train_loss = train_SCL(model, train_loader, criterion, optimizer, device, scl_weight)
    val_loss, val_accuracy = validate_SCL(model, valid_loader, criterion, device)

    if (epoch + 1) % 5 == 0:
        model_path = os.path.join(model_dir, f'(refined SCL) model_epoch_{epoch+1}.pth')
        torch.save(model.state_dict(), model_path)
        print(f'Model saved at: {model_path}')

Training Loss: 0.8302590346082728
Validation Loss: 1.4234132051467896, Accuracy: 4.666666666666667%
Training Loss: 0.7945339330967437
Validation Loss: 1.0747916877269745, Accuracy: 4.666666666666667%
Training Loss: 0.7968879678147904
Validation Loss: 0.8638421714305877, Accuracy: 4.666666666666667%
Training Loss: 0.7736934315651021
Validation Loss: 1.055364626646042, Accuracy: 4.666666666666667%
Training Loss: 0.7715306193270581
Validation Loss: 0.9156368315219879, Accuracy: 4.666666666666667%
Model saved at: /content/drive/MyDrive/ArtificialIntelligence/Project/Vit_model_save/(refined SCL) model_epoch_5.pth
Training Loss: 0.7744804233946698
Validation Loss: 0.8319910228252411, Accuracy: 4.666666666666667%
Training Loss: 0.7947372775128547
Validation Loss: 1.5643630027770996, Accuracy: 4.666666666666667%
Training Loss: 0.7738565510891854
Validation Loss: 1.2858258008956909, Accuracy: 4.666666666666667%
Training Loss: 0.7048082795548947
Validation Loss: 1.259290611743927, Accuracy: 16.6

## Test

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
from torchvision import transforms, datasets
from tqdm import tqdm
import pandas as pd

testset_path = '/content/drive/MyDrive/Project/test'

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

testset = datasets.ImageFolder(root=testset_path, transform=transform)
test_loader = DataLoader(testset, batch_size=1, shuffle=False)

## Inference

In [ ]:
if len(testset) == 500:
    print("The number of testset: 500")

    results = []
    model.eval()
    for images, _ in tqdm(test_loader, total=len(test_loader)):
        outputs = model(images)

        if 'logits' in outputs:
            logits = outputs['logits']
            _, predicted = torch.max(logits, 1)
            label = 'real' if predicted.item() == 1 else 'generated'
            results.append([label])
        else:
            print("Logits not found in model output.")

    df = pd.DataFrame(results, columns=['label'])
    results_path = '/content/drive/MyDrive/Project/ArtificialIntelligence/test/results_scl.csv'
    df.to_csv(results_path, index=False)
    print(f'Result stored: {results_path}')
else:
    print("Number of datasets is incorrect. Verified data count:", len(testset))


NameError: name 'testset' is not defined

In [ ]:
def calcul_accuracy(predicted_labels, labels_csv_path):
    true_labels_df = pd.read_csv(labels_csv_path, sep=';')
    true_labels = true_labels_df['label'].values
    predicted_labels_flat = [label[0] for label in predicted_labels]

    matches = true_labels == predicted_labels_flat
    correct_predictions = matches.sum()

    print(f"Correct predictions: {correct_predictions} out of {len(matches)}")

    accuracy = correct_predictions / len(matches)

    return accuracy

In [ ]:
predicted_labels = results
labels_csv_path = '/content/drive/MyDrive/Project/test/test_labels.csv'
accuracy = calcul_accuracy(predicted_labels, labels_csv_path)
print(f"Accuracy: {accuracy*100:.2f}%")

NameError: name 'calcul_accuracy' is not defined

##load model

In [ ]:
base_model = timm.create_model('vit_base_patch16_224', pretrained=False)
base_model.head = nn.Linear(base_model.head.in_features, num_classes)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["attn.qkv"],
    modules_to_save=["classifier"],
)

loaded_model = get_peft_model(base_model, lora_config)

model_path = '/content/drive/MyDrive/Project/Vit_model_save/Naive model_epoch_10.pth'
loaded_model.load_state_dict(torch.load(model_path), strict=False)
loaded_model.eval()

NameError: name 'timm' is not defined

In [ ]:
model = ModelWrapper(loaded_scl_model)

if len(testset) == 500:
    print("The number of testset: 500")

    results = []
    model.eval()
    for images, _ in tqdm(test_loader, total=len(test_loader)):
        outputs = model(images)

        if 'logits' in outputs:
            logits = outputs['logits']
            _, predicted = torch.max(logits, 1)
            label = 'real' if predicted.item() == 1 else 'generated'
            results.append([label])
        else:
            print("Logits not found in model output.")

    df = pd.DataFrame(results, columns=['label'])
    results_path = '/content/drive/MyDrive/Project/test/results_scl_2.csv'
    df.to_csv(results_path, index=False)
    print(f'Result stored: {results_path}')
else:
    print("Number of datasets is incorrect. Verified data count:", len(testset))

The number of testset: 500


100%|██████████| 500/500 [05:47<00:00,  1.44it/s]

Result stored: /content/drive/MyDrive/Project/test/results_scl_2.csv


In [ ]:
predicted_labels = results
labels_csv_path = '/content/drive/MyDrive/Project/test/test_labels.csv'
accuracy = calcul_accuracy(predicted_labels, labels_csv_path)
print(f"(SCL) Accuracy: {accuracy*100:.2f}%")

Correct predictions: 421 out of 500
(SCL) Accuracy: 84.20%


## Valid with generated image

In [ ]:
def validate(model, data_loader, criterion, device):
    model.eval()
    model.to(device)
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [ ]:
model = loaded_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
new_validation_loss, new_validation_accuracy = validate(model, extra_val_loader, criterion, device)

print(f'(Generated) Validation Loss: {new_validation_loss:.4f}')
print(f'(Generated) Validation Accuracy: {new_validation_accuracy:.2f}%')

NameError: name 'loaded_model' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

extra_val_path = '/content/drive/MyDrive/Project/GeneratedImages'
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

extra_val_dataset = datasets.ImageFolder(root=extra_val_path, transform=val_transform)
extra_val_indices = np.random.choice(len(extra_val_dataset), 100, replace=False)
extra_subset_val_dataset = Subset(extra_val_dataset, extra_val_indices)

class OneLabelDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label):
        self.dataset = dataset
        self.label = label

    def __getitem__(self, index):
        image, _ = self.dataset[index]
        return image, torch.tensor(self.label, dtype=torch.long)

    def __len__(self):
        return len(self.dataset)

extra_subset_val_dataset = OneLabelDataset(extra_subset_val_dataset, 0)

extra_val_loader = DataLoader(extra_subset_val_dataset, batch_size=32, shuffle=False)

def validate_SCL(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            logits = outputs['logits']
            loss = criterion(logits, labels)

            total_loss += loss.item()
            _, predicted = torch.max(logits, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(val_loader)
    accuracy = 100 * correct / total
    print(f'Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy}%')
    return avg_loss, accuracy

model = loaded_scl_model
model = ModelWrapper(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()

new_validation_loss, new_validation_accuracy = validate_SCL(model, extra_val_loader, criterion, device)

print(f'(Generated / SCL) Validation Loss: {new_validation_loss:.4f}')
print(f'(Generated / SCL) Validation Accuracy: {new_validation_accuracy:.2f}%')

Validation Loss: 0.6208, Accuracy: 88.0%
(Generated / SCL) Validation Loss: 0.6208
(Generated / SCL) Validation Accuracy: 88.00%
